In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [2]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist','landmark_dist_start','landmark_dist_end']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [3]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/Casual_RushMetro/landmarks.csv'
bs = set_data(file)

In [4]:
bs.columns

Index(['time_diff', 'temperature', 'humidity', 'windspeed', 'miles',
       'metro_dist', 'landmark_dist_start', 'landmark_dist_end', 'temp', 'hum',
       ...
       'weekday_0', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4',
       'weekday_5', 'weekday_6', 'weather_cat_1', 'weather_cat_2',
       'weather_cat_3'],
      dtype='object', length=303)

In [5]:
bs=bs.rename(columns = {'member_type_Registered':'member_type'})
bs = bs.drop(['temperature','windspeed','humidity'], 1)
bs.head()

,time_diff,miles,metro_dist,landmark_dist_start,landmark_dist_end,temp,hum,wind,rush_hour_1,member_type,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
663904,3.633,0.530361,0.537488,0.916964,0.401587,27.837500,63.5833,18.916579,1,1,...,0,1,0,0,0,0,0,1,0,0
244414,41.033,1.455686,0.020700,0.235437,0.839915,25.291651,67.7500,13.376014,0,0,...,0,0,0,0,1,0,0,1,0,0
770771,13.817,1.954768,0.210402,1.300777,0.235437,7.275000,61.3333,18.167586,1,1,...,0,0,0,1,0,0,0,1,0,0
114888,19.083,2.257878,0.135451,1.594631,0.255462,1.400000,59.0435,10.739832,1,1,...,0,0,1,0,0,0,0,1,0,0
426964,3.500,0.501247,0.023451,0.839915,1.300777,21.531651,90.2083,8.584375,0,0,...,0,0,0,0,1,0,0,0,1,0


In [6]:
X = bs.drop('member_type', 1)
y = bs['member_type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((914885, 299), (304962, 299), (914885,), (304962,))

### GridSearch Logistic Regression

In [7]:
n_estimators = 50
class_weight = [{0:.675, 1:.325}, {0:.7, 1:.3}, {0:.68, 1:.32}]
param_grid = {'class_weight':class_weight}

rf1 = RandomForestClassifier(n_estimators, n_jobs=-1)
rf_gs1 = GridSearchCV(rf1, param_grid, cv=3, scoring='roc_auc')
rf_gs1.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'class_weight': [{0: 0.675, 1: 0.325}, {0: 0.7, 1: 0.3}, {0: 0.68, 1: 0.32}]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [8]:
pred1 = rf_gs1.predict(X_test)
score1 = rf_gs1.score(X_test, y_test)
train_score1 = rf_gs1.score(X_train, y_train)

In [9]:
score1, train_score1

(0.86130262483181586, 0.99998991411225469)

In [10]:
confusion_matrix(y_test, pred1)

array([[ 27693,  22875],
       [ 10097, 244297]])

In [11]:
print(classification_report(y_test, pred1))

             precision    recall  f1-score   support

          0       0.73      0.55      0.63     50568
          1       0.91      0.96      0.94    254394

avg / total       0.88      0.89      0.89    304962



In [12]:
rf_gs1.best_params_

{'class_weight': {0: 0.7, 1: 0.3}}

In [13]:
roc_auc_score(y_test, pred1)

0.75397421073393445